In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers;
!pip install sentencepiece;
!pip install Ipywidgets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.7 MB/s eta 0:00:00


In [3]:
from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
)
import pickle
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai import tokenization_kobert
import json
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [4]:
class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [5]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/kobert_tokenizer_add/added_tokens.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=12290
mi=8002
for i in range(8002,12291):
    src_tokenizer.add_tokens(dict_swap[i])

In [6]:
model = EncoderDecoderModel.from_pretrained('leadawon/ossp-v0_3')
model.eval()
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id

Some weights of the model checkpoint at leadawon/ossp-v0_3 were not used when initializing EncoderDecoderModel: ['decoder.transformer.h.10.attn.masked_bias', 'decoder.transformer.h.9.attn.bias', 'decoder.transformer.h.2.attn.masked_bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.9.crossattention.masked_bias', 'decoder.transformer.h.1.crossattention.masked_bias', 'decoder.transformer.h.10.crossattention.masked_bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.0.attn.masked_bias', 'decoder.transformer.h.9.crossattention.bias', 'decoder.transformer.h.10.crossattention.bias', 'decoder.transformer.h.8.crossattention.masked_bias', 'decoder.transformer.h.5.crossattention.bias', 'decoder.transformer.h.4.crossattention.bias', 'decoder.transformer.h.2.crossattention.masked_bias', 'decoder.transformer.h.10.attn.bias', 'decoder.transformer.h.3.attn.masked_bias', 'decoder.transformer.h.6.cross

In [7]:
from IPython.display import display
import ipywidgets as widgets

dialect = widgets.Textarea(
    placeholder='번역할 제주방언',
    description="입력",
    disabled=False
)

button = widgets.Button(
    description='번역!',
    disabled=False,
    tooltip='방언을 번역합니다.'
)

standard = widgets.Textarea(
    description="출력",
    disabled=True
)

def translate(_):
    standard.value = ""
    text = dialect.get_interact_value()
    embeddings = src_tokenizer(text, return_attention_mask=False, return_token_type_ids=False, return_tensors='pt')
    output = model.generate(**embeddings)[0, 1:-1]
    standard.value = trg_tokenizer.decode(output.cpu())

button.on_click(translate)
display(dialect, button, standard)

Textarea(value='', description='입력', placeholder='번역할 제주방언')

Button(description='번역!', style=ButtonStyle(), tooltip='방언을 번역합니다.')

Textarea(value='', description='출력', disabled=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
